# <font color='blue'>ALPAR - Governo Digital - Processo Inteligente</font>

# <font color='red'>recomendação de ações - gravando em MySQL</font>

In [1]:
# !pip install mysql-connector-python

In [2]:
import pandas as pd
import numpy as np
import copy
import csv
import codecs
import time
from os.path import expanduser
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode

print("pandas versão", pd.__version__)
print("numpy versão", np.__version__)
print("csv versão", csv.__version__)

pd.options.display.max_rows = 2000
pd.options.display.width = 120
pd.options.display.max_colwidth = 100

pandas versão 1.1.3
numpy versão 1.19.2
csv versão 1.0


# <font color='black'>trata o dataset total</font>

In [3]:
My_host = 'localhost'
My_db_reco = 'bd_teste_reco'
My_db_fontes = 'bd_teste_fontes'
My_user = 'gd'
My_pw = 'Alpar@123'

In [4]:
sql_tasks = (
"SELECT "
"`Entidade`, "
"`Serviço`, "
"`Protocolo`, "
"`Data e Hora de conclusão`, "
"`Data e Hora de criação`, "
"`Ação`, "
"`Encaminhado para`, "
"`Processo encerrado`, "
"`Processo cancelado` "
"FROM tb_nx_tasks"
)

In [5]:
colunas = ['entidade', 'servico', 'protocolo', 'dthr_conclusao', 'dthr_criacao', 
           'acao', 'encaminhado_para', 'proc_encerrado', 'proc_cancelado']

try:
    connection = mysql.connector.connect(host = My_host, database = My_db_fontes, user = My_user, password = My_pw)
    
    df_tasks = pd.read_sql(sql_tasks, con=connection)
    df_tasks.columns = colunas
    
except mysql.connector.Error as error:
    print("Failed to read record from MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        connection.close()
        print(f'{df_tasks.shape[0]} registros lidos em {df_tasks.shape[1]} colunas')
        print("MySQL connection is closed")

3511896 registros lidos em 9 colunas
MySQL connection is closed


In [6]:
# transforma datas do tipo string para tipo data
df_tasks['dthr_conclusao']= pd.to_datetime(df_tasks['dthr_conclusao'])
df_tasks['dthr_criacao']= pd.to_datetime(df_tasks['dthr_criacao'])

In [8]:
# verifica a situação de cada protocolo
df = df_tasks[['protocolo', 'proc_encerrado', 'proc_cancelado']].copy()

df.loc[df['proc_encerrado'] == 'false', 'proc_encerrado'] = 0
df.loc[df['proc_encerrado'] == 'true', 'proc_encerrado'] = 1
df.loc[df['proc_cancelado'] == 'false', 'proc_cancelado'] = 0
df.loc[df['proc_cancelado'] == 'true', 'proc_cancelado'] = 1

df['proc_encerrado'] = pd.to_numeric(df['proc_encerrado'])
df['proc_cancelado'] = pd.to_numeric(df['proc_cancelado'])

# df

In [10]:
# agrupa por protocolo, obtem o máximo de encerrado e cancelado
df = df.groupby('protocolo').agg({'proc_encerrado': 'max', 'proc_cancelado': 'max'})

In [11]:
# filtra os protocolos cancelados
df.drop(df[df['proc_cancelado'] == 1].index, inplace=True)

# exclui a coluna de protocolos cancelados
df.drop('proc_cancelado', axis=1, inplace=True)

# cria um dataframe de processos encerrados
df_ence = df[(df['proc_encerrado'] == 1)].copy()

# cria um dataframe de processos em andamento
df_anda = df[(df['proc_encerrado'] == 0)].copy()

# exclui a coluna encerrado dos 2 dataframes
df_ence.drop('proc_encerrado', axis=1, inplace=True)
df_anda.drop('proc_encerrado', axis=1, inplace=True)

print(df_ence.shape)
print(df_anda.shape)

In [13]:
# cria o dataframe de encerrados e dataframe de em andamento
lst_ence = df_ence.index.tolist()
lst_anda = df_anda.index.tolist()

# <font color='black'>cria e trata o dataset de protocolos encerrados</font>

In [14]:
# cria dataset de tasks encerradas
df_ence = df_tasks[df_tasks['protocolo'].isin(lst_ence)].copy()

In [15]:
# coloca a string <vazio> nos valores NULL
lst_mod = list(df_ence[df_ence['encaminhado_para'].isnull()].index)
df_ence.loc[lst_mod, 'encaminhado_para'] = '<Vazio>'

In [16]:
# ordena por entidade, serviço, protocolo, dt_criação
df_ence.sort_values(by = ['entidade', 'servico', 'protocolo', 'dthr_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_ence.reset_index(drop = True, inplace = True)

In [17]:
# calcula a quantidade de dias da ação e exclui as datas de criação e de conclusão
df_ence['dias'] = (df_ence['dthr_conclusao'] - df_ence['dthr_criacao']) / np.timedelta64(1, 'D')
df_ence.drop(['dthr_conclusao', 'dthr_criacao'], axis=1, inplace=True)

In [18]:
# exclui colunas não utilizáveis
df_ence.drop(['proc_encerrado', 'proc_cancelado'], axis=1, inplace=True)

# df_ence

In [19]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['protocolo', 'entidade', 'servico'])
      .agg(
        lst_acao=('acao', lambda x: tuple(x)),
        lst_encaminhado=('encaminhado_para', lambda x: tuple(x)),
        sum_dias=('dias', sum)).reset_index())

In [20]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_ence = (df_ence.groupby(['entidade', 'servico', 'lst_acao', 'lst_encaminhado'])
      .agg(
        media_dias=('sum_dias', 'mean')).reset_index())

# <font color='black'>cria o dataset de protocolos em andamento</font>

In [22]:
# cria dataset de tasks em andamento
df_anda = df_tasks[df_tasks['protocolo'].isin(lst_anda)].copy()

In [23]:
# exclui colunas não utilizáveis
df_anda.drop(['proc_encerrado', 'proc_cancelado', 'dthr_conclusao'], axis=1, inplace=True)

# exclui as linhas com a coluna ação NULL
# df_anda = df_anda[~df_anda['acao'].isnull()].copy()     # esta linha é para quando ausência de dados for NULL
df_anda = df_anda[~(df_anda['acao'] == '')].copy()

# df_anda

In [24]:
# ordena por entidade, serviço, protocolo, dt_criação
df_anda.sort_values(by = ['entidade', 'servico', 'protocolo', 'dthr_criacao'], inplace = True)

# refaz o índice para a nova ordenação
df_anda.reset_index(drop = True, inplace = True)

In [25]:
# agrupa o dataframe por protocolo, entidade e serviço e cria uma tupla de ações como informação de coluna
df_anda = (df_anda.groupby(['protocolo', 'entidade', 'servico'])
      .agg(lst_acao=('acao', lambda x: tuple(x)))
      .reset_index()).copy()
df_anda.sort_values(['entidade', 'servico'], inplace = True)
# df_anda

# <font color='black'>gera estrutura de recomendações</font>

In [26]:
start = time.process_time()
tot = 0

# analisa cada protocolo em andamento
# procura a sequência do protocolo em andamento para sugerir o restante da sequência

# zera a lista de recomendações
lst_reco = []
const_max_tempo = 999999999.9
tot_anda = range(len(df_anda))
# tot_anda = range(1000)
print('total de protocolos a processar =', tot_anda)

# para cada protocolo em andamento procura uma coleção de sequências dos protocolos encerrados
anda_quebra = [None, None]
for i in tot_anda:
    if anda_quebra != [df_anda.loc[i, 'entidade'], df_anda.loc[i, 'servico']]:
        anda_quebra = [df_anda.loc[i, 'entidade'], df_anda.loc[i, 'servico']]
        df_aux1 = df_ence[(df_ence['entidade'] == df_anda.loc[i, 'entidade']) & 
                          (df_ence['servico'] == df_anda.loc[i, 'servico'])]
    
    n_acoes = len(df_anda.loc[i, 'lst_acao'])
    df_aux = df_aux1[(df_aux1['lst_acao'].apply(lambda x: x[:n_acoes]) == df_anda.loc[i, 'lst_acao'])]
    
    df_aux.reset_index(inplace = True)
    
    # zera a variável que vai achar o registro "entidade/serviço" com o menor tempo de conclusão
    # para quando for achado mais de 1 possibilidade
    min_tempo = const_max_tempo
    dt = {'sequencia': [], 'encaminhamento': []}

    qtd_casos = len(df_aux)
    tot += qtd_casos
        
    for k in range(qtd_casos):
        valor = round(np.float(df_aux.loc[k, 'media_dias']), 6)
        if (valor < min_tempo) or (min_tempo == None):
            min_tempo = valor
            dt = {'sequencia': list(df_aux.loc[k, 'lst_acao']),
                  'encaminhamento': list(df_aux.loc[k, 'lst_encaminhado'])}
            
    indice = len(list(df_anda.loc[i, 'lst_acao'])) if dt else -1
    if qtd_casos == 1:
        tipo_recomendacao = 2
    elif qtd_casos > 1:
        tipo_recomendacao = 3
    else:
        tipo_recomendacao = 1
        
    it = {'protocolo': df_anda.loc[i, 'protocolo'],
            'seq_atual': list(df_anda.loc[i, 'lst_acao']),
            'recomendacao': dt,
            'tempo_medio': None if min_tempo == const_max_tempo else min_tempo,
            'reco_acao_idx': indice,
            'qtd_casos' : qtd_casos,
            'tipo_recomendacao' : tipo_recomendacao}
    lst_reco.append(it)
    if i % 100 == 0:
        print(i, tot)

print(i, tot)
print('tempo =', time.process_time() - start)

total de protocolos a processar = range(0, 46157)
0 0
100 2582
200 6824
300 8321
400 11747
500 16578
600 19952
700 22255
800 27423
900 31108
1000 34063
1100 36705
1200 44199
1300 48272
1400 54447
1500 60740
1600 67361
1700 79161
1800 87872
1900 94127
2000 102981
2100 110467
2200 117985
2300 127449
2400 137290
2500 144642
2600 154623
2700 157005
2800 165819
2900 172661
3000 179553
3100 187076
3200 196427
3300 203111
3400 211856
3500 216583
3600 225337
3700 230686
3800 237771
3900 242918
4000 252206
4100 258622
4200 265813
4300 273043
4400 278569
4500 286771
4600 293574
4700 297902
4800 302045
4900 312280
5000 318114
5100 323353
5200 329839
5300 337218
5400 342522
5500 345166
5600 348956
5700 354296
5800 359225
5900 366464
6000 376269
6100 396850
6200 407207
6300 416382
6400 423710
6500 444255
6600 460278
6700 474145
6800 479061
6900 488281
7000 501757
7100 522167
7200 535896
7300 552207
7400 562700
7500 575693
7600 588183
7700 594139
7800 606928
7900 619771
8000 632434
8100 637400
8200 

# <font color='black'>cria um arquivo com recomendações</font>

## <font color='green'>define os SCRIPTS das tabelas de recomendações para o MySQL</font>

In [27]:
script_tb_recomenda = (
"CREATE TABLE `tb_recomenda` ("
  "`sg_acao`               int                 NOT NULL AUTO_INCREMENT,"
  "`nu_seq`                smallint            NOT NULL,"
  "`fk_protocolo`          varchar(21)         NOT NULL,"
  "`nm_acao`               varchar(128)        NOT NULL,"
  "`nm_encaminhado`        varchar(128)        NOT NULL,"
  "`fg_proximo`            tinyint             NOT NULL,"
  "PRIMARY KEY (`sg_acao`),"
  "KEY `fk_protocolo_idx` (`fk_protocolo`),"
  "CONSTRAINT `fk_protocolo` FOREIGN KEY (`fk_protocolo`) REFERENCES `tb_protocolo` (`pk_protocolo`)"
")"
)
script_tb_protocolo = (
"CREATE TABLE `tb_protocolo` ("
  "`pk_protocolo`          varchar(21)         NOT NULL,"
  "`tp_medio`              decimal(18,6)       DEFAULT NULL,"
  "`nm_prox_acao`          varchar(128)        DEFAULT NULL,"
  "`nm_prox_enc`           varchar(128)        DEFAULT NULL,"
  "`dt_timestamp`          timestamp           NOT NULL DEFAULT CURRENT_TIMESTAMP,"
  "`nu_qtd_casos`          int                 NOT NULL,"
  "`cd_tipo_reco`          smallint            NOT NULL,"
  "PRIMARY KEY (`pk_protocolo`)"
")"
)
tabelas = (('tb_protocolo', script_tb_protocolo), ('tb_recomenda', script_tb_recomenda))

## <font color='green'>CRIA as LISTAS de gravação de protocolos e recomendações no MySQL</font>

In [28]:
# prepara uma lista de tuplas para gravação da tabela "tb_protocolo" no MySQL
lst_protocolo = []
lst_recomenda = []
for k in lst_reco:
    prx_acao = None
    prx_encaminhamento = None
    
    n_acoes_ence = len(k['recomendacao']['sequencia'])
    for j in range(n_acoes_ence):
        if j == k['reco_acao_idx']:
            proximo = 1
            prx_acao = k['recomendacao']['sequencia'][j]
            prx_encaminhamento = k['recomendacao']['encaminhamento'][j]
        else:
            proximo = 0
        
        lst_recomenda.append((j, 
                              k['protocolo'], 
                              k['recomendacao']['sequencia'][j], 
                              k['recomendacao']['encaminhamento'][j], 
                              proximo
                             )
        )

    lst_protocolo.append((k['protocolo'], 
                          k['tempo_medio'], 
                          prx_acao, 
                          prx_encaminhamento,
                          k['qtd_casos'],
                          k['tipo_recomendacao']
                         )
    )

## <font color='green'>LIMPA as tabelas de protocolos e recomendações no MySQL</font>

In [29]:
try:
    connection = mysql.connector.connect(host = My_host, database = My_db_reco, user = My_user, password = My_pw)
    cursor = connection.cursor()
    
    # excluir tabelas
    for tb in reversed(tabelas):
        table_name = tb[0]
        table_script = tb[1]
        try:
            print("dropando tabela {}: \n".format(table_name), end='')
            apaga = xx = 'DROP TABLE IF EXISTS `' + My_db_reco + '`.`' + table_name + '`;'
            cursor.execute(apaga)
        except mysql.connector.Error as err:
            msg = 'já existe' if err.errno == errorcode.ER_TABLE_EXISTS_ERROR else err.msg
        else:
            print("OK")

    # incluir tabelas
    for tb in tabelas:
        table_name = tb[0]
        table_script = tb[1]
        try:
            print("criando tabela {}: \n".format(table_name), end='')
            cursor.execute(table_script)
        except mysql.connector.Error as err:
            print(err.errno, err.msg)
        else:
            print("OK")

except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

dropando tabela tb_recomenda: 
OK
dropando tabela tb_protocolo: 
OK
criando tabela tb_protocolo: 
OK
criando tabela tb_recomenda: 
OK
MySQL connection is closed


## <font color='green'>GRAVA as tabelas de protocolos e recomendações no MySQL</font>

In [30]:
try:
    connection = mysql.connector.connect(host = My_host, database = My_db_reco, user = My_user, password = My_pw)

    insert_protocolo = (
        "INSERT INTO tb_protocolo "
        "(pk_protocolo, tp_medio, nm_prox_acao, nm_prox_enc, nu_qtd_casos, cd_tipo_reco)"
        "VALUES (%s, %s, %s, %s, %s, %s)"
    )
    insert_recomenda = (
        "INSERT INTO tb_recomenda "
        "(nu_seq, fk_protocolo, nm_acao, nm_encaminhado, fg_proximo)"
        "VALUES (%s, %s, %s, %s, %s)"
    )
    
    cursor_p = connection.cursor()
    cursor_r = connection.cursor()


    cursor_p.executemany (insert_protocolo, lst_protocolo)
    cursor_r.executemany (insert_recomenda, lst_recomenda)

    connection.commit()

    print(cursor_p.rowcount, "Record inserted successfully into tb_protocolo table")
    print(cursor_r.rowcount, "Record inserted successfully into tb_recomenda table")


except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        cursor_p.close()
        cursor_r.close()
        connection.close()
        print("MySQL connection is closed")

46157 Record inserted successfully into tb_protocolo table
90313 Record inserted successfully into tb_recomenda table
MySQL connection is closed


# <font color='black'>TESTES</font>